In [1]:
import httpx

In [2]:
BASE_URL = "http://localhost:8000"
COLLECTION_NAME = "temp"

In [3]:
client = httpx.Client(base_url=BASE_URL)
print(f"Client set up to hit {BASE_URL}")

Client set up to hit http://localhost:8000


## Create collection

In [4]:
collection_payload = {
    "name": COLLECTION_NAME,
    "dimension": 384,
    "distance_metric": "cosine"
}

# Execute request
response = client.post("/v1/collections/", json=collection_payload)

# Print result
print(f"Status: {response.status_code}")
response.json()

Status: 201


{'name': 'temp', 'status': 'created', 'count': 0}

## Insert DataPoint

In [5]:
datapoint_payload = {
    "id": 1,
    "text": "Jupyter notebooks are great for interactive testing.",
    "metadata": {"source": "manual_test", "author": "dev"}
}

response = client.post(
    f"/v1/collections/{COLLECTION_NAME}/datapoints/", 
    json=datapoint_payload
)

print(f"Status: {response.status_code}")
response.json()

Status: 201


{'id': 1, 'status': 'inserted'}

## Insert Bulk

In [6]:
bulk_payload = [
    {
        "id": 2, 
        "text": "Deep Learning requires a lot of GPU power.", 
        "metadata": {"category": "hardware"}
    },
    {
        "id": 3, 
        "text": "Python is the primary language for AI.", 
        "metadata": {"category": "language"}
    }
]

response = client.post(
    f"/v1/collections/{COLLECTION_NAME}/datapoints/bulk", 
    json=bulk_payload
)

print(f"Status: {response.status_code}")
response.json()

Status: 201


{'inserted_count': 2}

## Inspect DataPoint

In [7]:
datapoint_id = 2

response = client.get(
    f"/v1/collections/{COLLECTION_NAME}/datapoints/{datapoint_id}"
)

print(f"Status: {response.status_code}")
# The notebook will pretty-print this dictionary automatically
response.json()

Status: 200


{'id': '2',
 'text': 'Deep Learning requires a lot of GPU power.',
 'metadata': {'category': 'hardware'}}

## Get DataPointEmbedding

In [8]:
response = client.get(
    f"/v1/collections/{COLLECTION_NAME}/datapoints/2/embedding"
)

data = response.json()
print(f"Vector ID: {data['id']}")
print(f"Vector (first 3 dims): {data['embedding'][:3]}") # Truncating for readability

Vector ID: 2
Vector (first 3 dims): [-0.014457735, -0.086457446, 0.04435644]


## Update DataPoint

In [9]:
update_payload = {
    "text": "Jupyter notebooks are great for interactive testing AND visualization.",
    "metadata": {"source": "manual_test", "status": "edited"}
}

response = client.put(
    f"/v1/collections/{COLLECTION_NAME}/datapoints/2", 
    json=update_payload
)

print(f"Status: {response.status_code}")
response.json()

Status: 200


{'id': 2, 'status': 'updated'}

## Delete DataPoint

In [10]:
del_resp = client.delete(f"/v1/collections/{COLLECTION_NAME}/datapoints/2")
print(f"Delete Point Status: {del_resp.status_code}")

Delete Point Status: 204


## Get Embedding

In [11]:
embed_payload = {
    "text": "What does Wärmedämmung mean?"
}

# 2. Send POST request
response = client.post("/v1/embedding/", json=embed_payload)

# 3. Handle response
print(f"Status: {response.status_code}")
data = response.json()

if response.status_code == 200:
    # Save the embedding for the search test later
    query_vector = data["embedding"]
    print(f"Received vector of length: {len(query_vector)}")
    print(f"First 5 dimensions: {query_vector[:5]}")
else:
    print(f"Error: {data}")

Status: 200
Received vector of length: 384
First 5 dimensions: [-0.2759299874305725, 0.2523435652256012, 0.07587683200836182, 0.67529296875, -0.36945605278015137]


## Get Search

In [12]:
search_payload = {
        "embedding": query_vector,
        "n_items": 2,              
    }

response = client.post(f"/v1/collections/{COLLECTION_NAME}/search/", json=search_payload)

print(f"Status: {response.status_code}")

if response.status_code == 200:
    results = response.json().get("results", [])
    print(f"Found {len(results)} matches.")
    for item in results:
        print(item)
else:
    print(f"Error: {response.text}")

Status: 200
Found 2 matches.
{'text': 'Python is the primary language for AI.', 'score': 0.041994855, 'metadata': {'id': 3, 'category': 'language'}}
{'text': 'Jupyter notebooks are great for interactive testing.', 'score': 0.022942796, 'metadata': {'id': 1, 'source': 'manual_test', 'author': 'dev'}}


## Get Ranking

In [13]:
ranking_payload = {
    "question": "What does Wärmedämmung mean?",
    "texts": [
        "Python is the primary language for AI.",          # Irrelevant
        "Jupyter notebooks are great for interactive testing.", # Irrelevant
    ]
}

response = client.post("/v1/ranking/", json=ranking_payload)

print(f"Status: {response.status_code}")
data = response.json()

# Print sorted results
print(f"Question: {data['question']}\n")
for item in data['results']:
    print(f"Score: {item['score']:.4f} | Text: {item['text']}")

Status: 200
Question: What does Wärmedämmung mean?

Score: -11.2252 | Text: Python is the primary language for AI.
Score: -11.3242 | Text: Jupyter notebooks are great for interactive testing.


## Full query

In [16]:
query_request = {
    "question": "What does Wärmedämmung mean?",
    
    # "Broad Search": Get 10 candidates from Qdrant (Fast)
    "n_retrieval": 10,  
    
    # "Precision Filter": Keep only the top 3 best matches after Cross-Encoding (Slow & Accurate)
    "n_ranking": 3,     
    
    # Optional: Override the default collection
    "collection_name": "temp" 
}

response = client.post("/v1/query/", json=query_request)

print(f"Status: {response.status_code}")
data = response.json()

# Print sorted results
print(f"Question: {data['question']}\n")
for item in data['results']:
    print(f"Score: {item['score']:.4f} | Text: {item['text']}")

Status: 200
Question: What does Wärmedämmung mean?

Score: -11.2252 | Text: Python is the primary language for AI.
Score: -11.3242 | Text: Jupyter notebooks are great for interactive testing.


## Delete collection

In [17]:
response = client.delete("/v1/collections/temp")

# Print result
print(f"Status: {response.status_code}")

try:
    data = response.json()
except ValueError:
    data = None  # no JSON returned

if data:
    print(data)

Status: 204


## Finalizing

In [18]:
client.close()